* creates `deriv/timing_df_recall.csv`
- make and save a dataframe containing onset and offset TRs for each cue presentation during recall

In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

import sklearn
import brainiak
import nilearn as nl
from nilearn import image, plotting, input_data

from scipy.spatial import distance

pd.options.display.max_rows = 200

- [SUB]_recall.log does not exist for all sub
- use [sub]_recall_mylog.log instead

### cols
- sub_num
- wed_id
- onset TR
- offset TR

In [64]:
def load_sub_df(sub_num):
  """ """
  TR_rate = 1.5
  # load sub log file
  fpath = 'data/behav/silvy_buckets/sub%.2iday2/%i_recall_mylog.log'%(100+sub_num,sub_num)
  f = open(fpath, "r")
  first_line = f.readline()
  # extract first TR tstamp
  first_tstamp = float(first_line.split()[1])
  # init df
  sub_df = pd.DataFrame(columns=['sub_num','wed_id','onset_TR','offset_TR'])
  # loop over rows in logfile (stim onsets)
  for x in f:
    # extract info
    RV,tstamp,stim = x.split(' ')
    tstamp = float(tstamp[:-1])
    wed_id = stim.split('/')[-1].split('.')[0][1:]
    onset_TR = np.floor((tstamp-first_tstamp)/TR_rate)
    # populate df with info
    sub_df.loc[tstamp,'sub_num'] = sub_num
    sub_df.loc[tstamp,'wed_id'] = wed_id
    sub_df.loc[tstamp,'onset_TR'] = onset_TR
  def include_offset_TR_col(sub_df):
    sub_df.iloc[:-1].loc[:,'offset_TR'] = sub_df.iloc[1:].onset_TR.values
    sub_df.iloc[-1].offset_TR = -1
    return sub_df
  sub_df = include_offset_TR_col(sub_df)
  return sub_df


sub_dfL = [] 
for sub_num in range(1,45):
  try:
    sub_df = load_sub_df(sub_num)
  except:
    print('error sub=',sub_num)
    continue
  sub_dfL.append(sub_df)



error sub= 1
error sub= 16
error sub= 20
error sub= 21


//anaconda3/envs/brain/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
timing_df = pd.concat(sub_dfL)
timing_df.index = np.arange(len(timing_df))
